## Import Libraries

In [ ]:
!pip install selenium
!pip install selenium webdriver-manager

In [ ]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import re
import time
import random
import os
from selenium import webdriver


import sys

### Define some variables


In [ ]:
BASE_URL = "https://nomfoundation.org"

### Get the page

In [ ]:
respond = requests.get(BASE_URL)
soup = BeautifulSoup(respond.text, 'html.parser')

In [ ]:
# nom_texts_html  = soup.find_all(class_='has_submenu').text == 'Nôm Texts'
nom_texts_html = [a.find_next('ul') for a in soup.find_all('a', class_='has_submenu') if a.text == 'Nôm Texts'][0]

In [ ]:
nom_texts_html
links = [(a.text, a['href']) for a in nom_texts_html.find_all('a')]
print(links)
links_dict = {text: BASE_URL + href[: href.find('?')] if href.startswith('/') else href[: href.find('?') - 1] for text, href in links}
for text, link in links_dict.items():
    print(text, link)

[('Thắng Nghiêm Temple', 'http://lib.nomfoundation.org/collection/2/?uiLang=en'), ('Phổ Nhân Temple', 'http://lib.nomfoundation.org/collection/3/?uiLang=en'), ('Digital Library of Hán-Nôm', '/nom-project/Digital-Library-of-Han-Nom?uiLang=en'), ('Tale of Kiều', '/nom-project/Tale-of-Kieu?uiLang=en'), ('Lục Vân Tiên', '/nom-project/Luc-Van-Tien?uiLang=en'), ('Chinh Phụ Ngâm Khúc', '/nom-project/Chinh-Phu-Ngam-Khuc?uiLang=en'), ('Hồ Xuân Hương', '/nom-project/Ho-Xuan-Huong?uiLang=en'), ('History of Greater Vietnam', '/nom-project/History-of-Greater-Vietnam?uiLang=en')]
Thắng Nghiêm Temple http://lib.nomfoundation.org/collection/2
Phổ Nhân Temple http://lib.nomfoundation.org/collection/3
Digital Library of Hán-Nôm https://nomfoundation.org/nom-project/Digital-Library-of-Han-Nom
Tale of Kiều https://nomfoundation.org/nom-project/Tale-of-Kieu
Lục Vân Tiên https://nomfoundation.org/nom-project/Luc-Van-Tien
Chinh Phụ Ngâm Khúc https://nomfoundation.org/nom-project/Chinh-Phu-Ngam-Khuc
Hồ Xuân H

## Set up crawl data for each page

### crawl in History of Greater Vietnam

In [ ]:
history_base_url = links_dict['History of Greater Vietnam']
history_url = history_base_url + '/Fulltext?uiLang=en'
history_url

'https://nomfoundation.org/nom-project/History-of-Greater-Vietnam/Fulltext?uiLang=en'

In [ ]:
respond = requests.get(history_url)
soup = BeautifulSoup(respond.text, 'html.parser')

In [ ]:
# get all the links to the chapters
chapters = soup.find_all('div', class_='submenu')

In [ ]:
chapters = []
for item in soup.find_all("div", class_="item"):
    link = item.find("a")["href"]  # Lấy đường dẫn từ thẻ <a>
    name = item.find("a").text.strip()  # Lấy tên từ nội dung văn bản
    if link != '#':
      chapters.append({"link": history_base_url + '/' + link, "name": name})

# Print results
for chapter in chapters:
    print(f"Link: {chapter['link']}, Name: {chapter['name']}")

Link: https://nomfoundation.org/nom-project/History-of-Greater-Vietnam/Fulltext/100-Tuc-bien-tu?uiLang=en, Name: Tục biên tự
Link: https://nomfoundation.org/nom-project/History-of-Greater-Vietnam/Fulltext/101-Tuc-bien-thu?uiLang=en, Name: Tục biên thư
Link: https://nomfoundation.org/nom-project/History-of-Greater-Vietnam/Fulltext/102-Ngoai-ky-toan-thu?uiLang=en, Name: Ngoại kỷ toàn thư
Link: https://nomfoundation.org/nom-project/History-of-Greater-Vietnam/Fulltext/103-Toan-thu-bieu?uiLang=en, Name: Toàn thư biểu
Link: https://nomfoundation.org/nom-project/History-of-Greater-Vietnam/Fulltext/104-Toan-thu-pham-le?uiLang=en, Name: Toàn thư phàm lệ
Link: https://nomfoundation.org/nom-project/History-of-Greater-Vietnam/Fulltext/105-Ky-nien-muc-luc?uiLang=en, Name: Kỷ niên mục lục
Link: https://nomfoundation.org/nom-project/History-of-Greater-Vietnam/Fulltext/106-Khao-tong-luan?uiLang=en, Name: Khảo tổng luận
Link: https://nomfoundation.org/nom-project/History-of-Greater-Vietnam/Fulltext/1-K

Use selenium to emulate a web browser


In [ ]:
name, link = chapters[0]['name'], chapters[0]['link']

In [ ]:
# download and install the latest version of ChromeDriver
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i google-chrome-stable_current_amd64.deb
!apt-get -f install -y 

# download and install the latest version of ChromeDriver
!wget https://chromedriver.storage.googleapis.com/$(google-chrome --version | awk '{print $3}' | cut -d '.' -f 1-3)/chromedriver_linux64.zip
!unzip chromedriver_linux64.zip
!chmod +x chromedriver
!mv chromedriver /usr/local/bin/


--2024-12-17 08:17:28--  https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
Resolving dl.google.com (dl.google.com)... 108.177.119.91, 108.177.119.190, 108.177.119.136, ...
Connecting to dl.google.com (dl.google.com)|108.177.119.91|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 112421156 (107M) [application/x-debian-package]
Saving to: ‘google-chrome-stable_current_amd64.deb.2’

google-chrome-stabl 100%[===================>] 107.21M   306MB/s    in 0.3s    

2024-12-17 08:17:28 (306 MB/s) - ‘google-chrome-stable_current_amd64.deb.2’ saved [112421156/112421156]

(Reading database ... 123954 files and directories currently installed.)
Preparing to unpack google-chrome-stable_current_amd64.deb ...
Unpacking google-chrome-stable (131.0.6778.139-1) over (131.0.6778.139-1) ...
Setting up google-chrome-stable (131.0.6778.139-1) ...
Processing triggers for man-db (2.10.2-1) ...
Reading package lists... Done
Building dependency tree... Done
Read

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.common.exceptions import WebDriverException, TimeoutException, NoSuchElementException


# Thiết lập trình duyệt
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)


In [ ]:
def get_number_pages(html_soup):
    pages = html_soup.find('p', class_='title_hd')
    text = pages.text
    number_pages = int(re.search(r'\d+', text).group())
    return number_pages

In [ ]:
def extract_sentences_from_html(new_soup):
    # get the main content
    main_html = new_soup.find('td', {'align': 'justify', 'width': '60%', 'valign': 'top'})

    if main_html is None:
        return []

    text = main_html.text
    cleaned_text = re.sub(r'\[.*?\]', '', text)
    sentences = re.split(r'(?<=\.)', cleaned_text)

    final_sentences = []

    for sentence in sentences:
        sentence = sentence.strip()

        if sentence and sentence != ".":
            final_sentences.append([sentence])

    return final_sentences


In [ ]:
def create_driver():
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')

    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    return driver

In [ ]:
def scrape_all_pages(url):
    all_data = []

    attempt = 0
    max_attempts = 5  
    page_data = []
    driver = create_driver()

    while attempt < max_attempts:
        try:
            print(f"Attempt {attempt + 1}/{max_attempts} to load the page: {url}")

            driver.get(url)
            html = driver.page_source

            new_soup = BeautifulSoup(html, 'html.parser')

            # get the number of pages
            total_pages = get_number_pages(new_soup)
            print("Total pages:", total_pages)

            # load all pages
            for page_num in range(total_pages):
                print(f"Processing page {page_num + 1}/{total_pages}")
                # Retry if the page fails to load
                attempt_page = 0
                while attempt_page < 3: 
                    try:
                        page_link = driver.find_element(By.XPATH, f'//a[@href="javascript:GotoPage({page_num})"]')
                        page_link.click()

                        time.sleep(random.randint(1, 3))

                        html = driver.page_source
                        new_soup = BeautifulSoup(html, 'html.parser')

                        page_data = extract_sentences_from_html(new_soup)

                        all_data.extend(page_data)
                        break

                    except (WebDriverException, TimeoutException, NoSuchElementException) as e:
                        print(f"Error on attempt {attempt_page + 1} for page {page_num + 1}: {str(e)}")
                        attempt_page += 1
                        if attempt_page < 3:
                            print("Retrying page...")
                            time.sleep(random.randint(3, 5))
                        else:
                            print(f"Max retries reached for page {page_num + 1}. Skipping this page.")
                            break  # Nếu hết retries, bỏ qua trang hiện tại

            break

        except (WebDriverException, TimeoutException, Exception) as e:
            print(f"Error on attempt {attempt + 1} to load the page: {str(e)}")
            attempt += 1
            if attempt < max_attempts:
                print("Retrying to load the page...")
                time.sleep(random.randint(3, 5))
            else:
                print("Max retries reached. Unable to load the page.")
                break  
    return all_data

Process for all link in list 

In [ ]:
#tạo một folder chứa file
folder_name = 'nom_texts'
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

In [ ]:
chapter = chapters[55]
name, link = chapter['name'], chapter['link']
print(name, link)

Tương Dực Đế https://nomfoundation.org/nom-project/History-of-Greater-Vietnam/Fulltext/65-Tuong-Duc-De?uiLang=en


In [ ]:
for chapter in chapters[55:]:
    name, link = chapter['name'], chapter['link']
    print(name, link)
    final_sentences = scrape_all_pages(link)

    # save to folder_name
    name = name.replace(' ', '_')
    print(name)
    # lưu vào file txt với tên theo
    with open(f'{folder_name}/{name}.txt', 'w') as f:
        for sentence in final_sentences:
            f.write(sentence[0] + '\n')
    time.sleep(random.randint(5, 10))


Tương Dực Đế https://nomfoundation.org/nom-project/History-of-Greater-Vietnam/Fulltext/65-Tuong-Duc-De?uiLang=en
Attempt 1/5 to load the page: https://nomfoundation.org/nom-project/History-of-Greater-Vietnam/Fulltext/65-Tuong-Duc-De?uiLang=en
Total pages: 66
Processing page 1/66
Processing page 2/66
Processing page 3/66
Processing page 4/66
Processing page 5/66
Processing page 6/66
Processing page 7/66
Processing page 8/66
Processing page 9/66
Processing page 10/66
Processing page 11/66
Processing page 12/66
Processing page 13/66
Processing page 14/66
Processing page 15/66
Processing page 16/66
Processing page 17/66
Processing page 18/66
Processing page 19/66
Processing page 20/66
Processing page 21/66
Processing page 22/66
Processing page 23/66
Processing page 24/66
Processing page 25/66
Processing page 26/66
Processing page 27/66
Processing page 28/66
Processing page 29/66
Processing page 30/66
Processing page 31/66
Processing page 32/66
Processing page 33/66
Processing page 34/66
Pr